In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import accuracy_score
import operator
from timeit import default_timer as timer
from foot_challenge.src.main import convert_to_one_hot
from foot_challenge.src.main import prepare_attribut_data, prepare_result_data
from foot_challenge.src.main import DRAW, HOME_WINS, AWAY_WINS

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
#HYPER_PARAMS:
scaling=True
test_size=0.1978
max_iter_list = [1000] # Last one used was 7000000
solver_list = ['lbfgs','sag']  # sag and saga take a lot of time
regularization_factor_list = [ 0.000098,0.0001,0.0005,0.000112,0.0001123454,0.00019254,0.000192,0.000193,0.000197,0.0002,0.00025]

In [3]:

train_path_away = '../data/Train_Data/train_away_team_statistics_df.csv'
train_path_home = '../data/Train_Data/train_home_team_statistics_df.csv'
train_path_result = '../data/Train_Data/Y_train.csv'
df_attribut = prepare_attribut_data(train_path_away, train_path_home,use_player_data=True).drop(['ID'], axis=1)
df_result = prepare_result_data(train_path_result)

In [4]:
df_attribut.head()

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,HOME_PLAYER_STARTING_LINEUP_5_last_match_std,HOME_PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std,HOME_PLAYER_TACKLES_5_last_match_std,HOME_PLAYER_TOTAL_CROSSES_5_last_match_std,HOME_PLAYER_TOTAL_DUELS_5_last_match_std,HOME_PLAYER_YELLOWCARDS_5_last_match_std,HOME_PLAYER_PUNCHES_5_last_match_std,HOME_PLAYER_LONG_BALLS_5_last_match_std,HOME_PLAYER_LONG_BALLS_WON_5_last_match_std,HOME_PLAYER_SHOTS_OFF_TARGET_5_last_match_std
0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,3.0,6.0,...,623.0,409.0,431.0,250.0,666.0,710.0,0.0,0.0,0.0,0.0
1,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,6.0,8.0,...,63.0,180.0,245.0,156.0,397.0,357.0,0.0,0.0,0.0,0.0
2,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,2.0,7.0,...,1180.0,286.0,527.0,252.0,774.0,805.0,0.0,0.0,0.0,0.0
3,7.0,5.0,5.0,6.0,6.0,9.0,9.0,2.0,2.0,0.0,...,357.0,159.0,344.0,307.0,353.0,280.0,0.0,0.0,0.0,0.0
4,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,4.0,7.0,...,329.0,327.0,414.0,262.0,376.0,248.0,0.0,0.0,0.0,0.0


In [5]:
df_result.head()

,result
0,-2
1,-1
2,-2
3,1
4,-1


In [6]:

attribut_train, attribut_test, result_train, result_test = train_test_split(df_attribut, df_result, test_size=test_size,
                                                                            random_state=25)


In [7]:
result_test

,result
2167,-1
6662,-1
2280,-1
3968,1
11918,-2
...,...
673,-1
8105,1
7216,-2
6576,-1


#Cell changed to markdown, to not be executed
min_index=0
max_index=20000
for i in [1,2]:   

    column_1=attribut_test.columns[i]
    column_2=attribut_test.columns[i+140]
    plt.figure(figsize=(16, 10))
    plt.scatter(attribut_test.loc[(result_test == DRAW).values, column_1].iloc[min_index:max_index], attribut_test.loc[(result_test == DRAW).values, column_2].iloc[min_index:max_index], label="DRAW", color='gray')

    plt.xlabel(column_1)
    plt.ylabel(column_2)
    plt.title("Train Set")
    plt.legend()
    plt.show()

In [8]:
if scaling:
    scaler = StandardScaler()
    attribut_train = scaler.fit_transform(attribut_train)
    attribut_test = scaler.transform(attribut_test)

In [9]:
attribut_train

array([[-0.78467648, -0.58864515, -0.89347187, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.24278434,  0.70598719,  0.5357066 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.0997026 , -1.23596132,  0.17841198, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.58527128,  0.05867102, -0.17888263, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.24278434,  0.70598719, -0.53617725, ...,  0.        ,
         0.        ,  0.        ],
       [-1.46965037, -0.58864515, -0.89347187, ...,  0.        ,
         0.        ,  0.        ]], shape=(9869, 884))

In [ ]:

from sklearn.linear_model import LogisticRegression


accuracy_dict = {}
time_dict = {}
LR_dict = {}
for max_iter in max_iter_list:
    for solver in solver_list:
        for regularization_factor in regularization_factor_list:
            start = timer()

            LR = LogisticRegression(max_iter=max_iter, solver=solver, C=regularization_factor,tol=1e-40)
            LR.fit(attribut_train,result_train)
            yhat = LR.predict(attribut_test)
            accuracy = np.round(accuracy_score(result_test, yhat), 4)
            key = f'solver_{solver}_iter_{max_iter}_C_{regularization_factor}'
            accuracy_dict[key] = accuracy
            LR_dict[key] = LR

            end = timer()
            print(f'for this hyperparam {key} the training took ')

            delta = end - start
            minutes = round(delta // 60)
            delta %= 60
            seconds = round(delta)
            time_str = f"{minutes:02d}m{seconds:02d}s"
            time_dict[key] = time_str
            print(time_str)



In [ ]:
accuracy_dict

In [ ]:
max_accuracy = max(accuracy_dict.items(), key=operator.itemgetter(1))
LR_best = LR_dict[max_accuracy[0]]

In [ ]:
time_dict

In [ ]:
max_accuracy

In [ ]:
yhat = LR_best.predict(attribut_test)
np.round(accuracy_score(result_test, yhat), 4)

In [ ]:
yhat

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(result_test, yhat)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[-2, -1, 1], )
disp.plot(cmap="Blues")

In [ ]:
test_path_away = '../data/Test_Data/test_away_team_statistics_df.csv'
test_path_home = '../data/Test_Data/test_home_team_statistics_df.csv'
df_attribut_test = prepare_attribut_data(test_path_away, test_path_home,use_player_data=True,is_test=True)
df_attribut_test_id_dropped=df_attribut_test.drop(['ID'], axis=1)
if scaling:
    df_attribut_test_id_dropped = scaler.transform(df_attribut_test_id_dropped)

In [ ]:
df_attribut_test

In [ ]:
yhat_test = LR_best.predict(df_attribut_test_id_dropped)

In [ ]:
y_pred_test = convert_to_one_hot(yhat_test)

In [ ]:
test_prediction = pd.concat([df_attribut_test['ID'], y_pred_test], axis=1)

In [ ]:
test_prediction.to_csv('../submission/logistic_benchmark_submission.csv', index=False)